In [1]:
import tflearn 
import scrapy
import requests
import urllib
import string
import os
import matplotlib.pyplot as plt
import nltk
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import random
import re
import nltk.corpus
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess
from nltk.stem.lancaster import LancasterStemmer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow import keras
from nltk.util import pr
from nltk.corpus import stopwords
from textblob import TextBlob
nltk.download('stopwords')

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\csgoc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
sad=os.path.exists('./scrapped_data.txt')
if (sad == True) : 
    os.remove("scrapped_data.txt")

In [3]:
with open("cons.txt",'w') as f:
   pass
with open("pros.txt",'w') as o:
    pass
with open("neut.txt",'w') as u:
    pass
f.close()
o.close()
u.close()

In [4]:
df = pd.read_csv ('training_data_set_final.csv')

In [5]:
df['input'] = df['input'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
df['input'] = df['input'].apply(lambda wrd: ''.join(wrd))

In [6]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(df['input'])
train = tokenizer.texts_to_sequences(df['input'])
x_train = pad_sequences(train)
le = LabelEncoder()
y_train = le.fit_transform(df['tag'])

In [7]:
input_shape = x_train.shape[1]

In [8]:
vocabulary = len(tokenizer.word_index)
output_length = le.classes_.shape[0]

In [9]:
#creating the model

i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1, 10)(i)
x = LSTM(10, return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length, activation="softmax")(x)
#model = Model(i, x)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
#model.compile(loss="sparse_categorical_crossentropy",optimizer='adam', metrics=['accuracy'])

In [11]:
#train = model.fit(x_train, y_train,epochs = 80)

In [12]:
#model.save("saved_model.h5")

In [13]:
model = keras.models.load_model('saved_model.h5')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
links=[]
try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
print("enter topic to debate")
user_response = input()
first_line=user_response
user_response=user_response + " is good or bad ?"
for j in search(user_response, tld="co.in", num=30, stop=30, pause=2):\
    links.append(j)    

enter topic to debate
death sentence 


In [15]:
class WikiScrapeSpider(scrapy.Spider):
    name = 'wiki_scrape'
    #allowed_domains = ['www.wikipedia.org']
    start_urls = []
    for link in links : 
        start_urls.append(link)
    def parse(self, response):
        datas = response.xpath('//p').extract()
        for item in datas:
            all_items = {
                'datas' : BeautifulSoup(item).text
                 
            }
            yield all_items

process = CrawlerProcess(settings={
    "FEEDS": {
        "scrapped_data.txt": {"format": "json"},
    },
})
process.crawl(WikiScrapeSpider)
process.start()
    

    
    

2021-12-20 16:01:32 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-12-20 16:01:32 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.5, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.7.7 (tags/v3.7.7:d7c567b08f, Mar 10 2020, 10:41:24) [MSC v.1900 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 35.0.0, Platform Windows-10-10.0.19041-SP0
2021-12-20 16:01:32 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-12-20 16:01:32 [scrapy.crawler] INFO: Overridden settings:
{}
2021-12-20 16:01:32 [scrapy.extensions.telnet] INFO: Telnet Password: 835097a14a502e1e
2021-12-20 16:01:32 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-12-20 16:01:32 [scrapy.middleware] INFO: Enabled downloader middlewares:
['s

2021-12-20 16:01:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.thehindu.com/opinion/op-ed/is-it-time-to-abolish-the-death-penalty/article25735508.ece>
{'datas': '\n'}
2021-12-20 16:01:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.thehindu.com/opinion/op-ed/is-it-time-to-abolish-the-death-penalty/article25735508.ece>
{'datas': 'The death penalty has been criticised for far too long without an understanding of its nuances. It is criticised mainly on three counts: arbitrariness, irreversibility and human rights. However, the punishment passes muster on all accounts. Its constitutionality has not only been upheld in India but also in the bastion of liberal democracy that is the U.S. The retention of the death penalty is not a reflection of “uncivilised” polity in theocratic states that have come to be defined by violence but a creation of the individual geopolitical circumstances of each state.'}
2021-12-20 16:01:32 [scrapy.core.scraper] DEBUG: Scraped from 

2021-12-20 16:01:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.thehindu.com/opinion/op-ed/is-it-time-to-abolish-the-death-penalty/article25735508.ece>
{'datas': 'In 2015, the Law Commission called for abolition of the death penalty for ordinary crimes, and activists continue to argue for abolishing it altogether. Political will in India is still bound by populism. However, the constitutionality of the death penalty will continue to be challenged and, sooner or later, the Supreme Court will have to answer whether absence of political will is sufficient ground to override the right to life.'}
2021-12-20 16:01:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.thehindu.com/opinion/op-ed/is-it-time-to-abolish-the-death-penalty/article25735508.ece>
{'datas': 'Avi Singh is an advocate who is the Additional Standing Counsel for criminal cases for the Government of the NCT of Delhi'}
2021-12-20 16:01:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.thehi

2021-12-20 16:01:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.britannica.com/topic/capital-punishment/Arguments-for-and-against-capital-punishment>
{'datas': 'In 1971 the United Nations General Assembly passed a resolution that, “in order fully to guarantee the right to life, provided for in…the Universal Declaration of Human Rights,” called for restricting the number of offenses for which the death penalty could be imposed, with a view toward abolishing it altogether. This resolution was reaffirmed by the General Assembly in 1977. Optional protocols to the European Convention on Human Rights (1983) and to the International Covenant on Civil and Political Rights (1989) have been established, under which countries party to the convention and the covenant undertake not to carry out executions. The Council of Europe (1994) and the EU (1998) established as a condition of membership in their organizations the requirement that prospective member countries suspend executions 

2021-12-20 16:01:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.aclu.org/other/case-against-death-penalty>
{'datas': "Despite the Supreme Court's 1976 ruling in Gregg v. Georgia, et al, the ACLU continues to oppose capital punishment on moral, practical, and constitutional grounds:"}
2021-12-20 16:01:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.aclu.org/other/case-against-death-penalty>
{'datas': 'Capital punishment is cruel and unusual. It is cruel because it is a relic of the earliest days of penology, when slavery, branding, and other corporal punishments were commonplace. Like those barbaric practices, executions have no place in a civilized society. It is unusual because only the United States of all the western industrialized nations engages in this punishment.\xa0 It is also unusual because only a random sampling of convicted murderers in the United States receive a sentence of death.'}
2021-12-20 16:01:33 [scrapy.core.scraper] DEBUG: Scraped from

...

2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'According to data gathered by The Marshall Project and Associated Press, 1571 prisoners in the United States had died of COVID-19 as of December 10, 2020, exceeding the total number of executions performed in the modern era of the death penalty (1529). Deaths from COVID-19 among death row inmates rivaled the number of executions this year, with at least 16 confirmed cases of death row prisoners who died from COVID. '}
2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'Alfonso Salazar'}
2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/th

2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'At the national level, the Biden-Harris ticket pledged to reduce mass incarceration, address racial bias in the legal system, and eliminate the federal death penalty. Citing the prevalence of wrongful convictions, Biden’s platform says, “Because we cannot ensure we get death penalty cases right every time, Biden will work to pass legislation to eliminate the death penalty at the federal level, and incentivize states to follow the federal government’s example.”'}
2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': '2020 brought a national reckoning on racial justice that was sparked by outrage at police and white vigilante killings of Bla

2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'The Court itself issued very few merits decisions in capital cases in 2020:'}
2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'James McKinney'}
2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'In a 5-4 vote along partisan lines, the Court upheld James Erin McKinney’s death sentence after the Arizona Supreme Court reweighed the aggravating and mitigating evidence in his case without affording him a new sentencing trial. McKinney was sentenced to death in 1993 under Arizona’s judge-o

2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'In November, after Justice Barrett’s confirmation, the Court intervened to vacate a stay of execution issued by the district court that would have temporarily halted Orlando Hall’s federal execution. The increasingly polarized Court did not even wait for the circuit court to rule on the government’s request to vacate the district court’s injunction, granting the government’s application and vacating the district court’s stay without explanation. Justices Stephen Breyer, Sonia Sotomayor, and Elena Kagan dissented.'}
2021-12-20 16:01:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://deathpenaltyinfo.org/facts-and-research/dpic-reports/dpic-year-end-reports/the-death-penalty-in-2020-year-end-report>
{'datas': 'Cover of the Racist Roots report'}
2021-12-20 16:01:43 [scrapy.core.scr

2021-12-20 16:01:43 [scrapy.core.engine] INFO: Closing spider (finished)
2021-12-20 16:01:43 [scrapy.extensions.feedexport] INFO: Stored json feed (2086 items) in: scrapped_data.txt
2021-12-20 16:01:43 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 7871,
 'downloader/request_count': 30,
 'downloader/request_method_count/GET': 30,
 'downloader/response_bytes': 1545576,
 'downloader/response_count': 30,
 'downloader/response_status_count/200': 29,
 'downloader/response_status_count/403': 1,
 'elapsed_time_seconds': 11.22536,
 'feedexport/success_count/FileFeedStorage': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2021, 12, 20, 10, 31, 43, 645878),
 'httpcompression/response_bytes': 5166239,
 'httpcompression/response_count': 27,
 'httperror/response_ignored_count': 1,
 'httperror/response_ignored_status_count/403': 1,
 'item_scraped_count': 2086,
 'log_count/DEBUG': 2116,
 'log_count/ERROR': 1,
 'log_count/INFO': 12,
 'response_rec

In [16]:
data=[]
n_lin=[]
final_data=[]
with open('scrapped_data.txt') as file:
    for line in file:
        input_line=line.rstrip()
        input_line = input_line[11:]
        input_line = re.sub(r'\\[u]\S\S\S\S[s]', "", input_line)
        input_line = re.sub(r'\\[u]\S\S\S\S', "", input_line)
        data.append(input_line)

textfile = open("mid-cleaned_data.txt", "w")
for element in data:
    textfile.write(element + "\n")

with open('mid-cleaned_data.txt') as file:
    for line in file:
        input_line=line.rstrip()
        input_line = input_line.replace('.','\n')
        n_lin.append(input_line)

textfile = open("new_line.txt", "w")
for element in n_lin:
    textfile.write(element + "\n")

with open('new_line.txt') as file:
    for line in file:
        input_line=line.rstrip()
        input_line = re.sub(r"(@[A-Za-z0-9]+ |)|([^0-9A-Za-z | ,| \ \t])|^rt|http+?", "", input_line)
        if(len(input_line) > 55):
            final_data.append(input_line+'.')
textfile = open("cleaned_data.txt", "w")
for element in final_data:
    textfile.write(element + "\n")



In [17]:
sad=os.path.exists('./mid-cleaned_data.txt')
if (sad == True) : 
    os.remove("mid-cleaned_data.txt")
sad=os.path.exists('./new_line.txt')
if (sad == True) : 
    os.remove("new_line.txt")

In [18]:
f=open('cleaned_data.txt','r',errors = 'ignore') 
raw_doc = f.read()

In [19]:
sent_tokens = nltk.sent_tokenize(raw_doc) 
word_tokens = nltk.word_tokenize(raw_doc)

In [20]:
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens (tokens): 
  return [lemmer.lemmatize(token) for token in tokens] 
def LemNormalize(text): 
  return LemTokens (nltk.word_tokenize(text.lower()))
sent_tokens = nltk.sent_tokenize(raw_doc) 

In [21]:
GREET_INPUTS = ("hello", "hii", "hi", "hey", "good morning","morning", "gm","whatsup","hy","how you doing ?","How are you ?")
GREET_RESPONSES = ["hii, DebaterX can see a beautifull face","hey their", "hello there","Good to see you mate, I'm DebaterX","Let's have a good converstation","Lets have a fruitfull debate ","What's on your mind ? , DebaterX is very curious !","I can see you have a good idea inside you ! Tell me !, DebaterX Can't Wait !",]
Extra_greet="name"
res=["My name is DebaterX","I'm DebaterX","People call me DebaterX"]
def greet(sentence):
    for t in sentence.split():
        if(t==Extra_greet):
            return random.choice(res)
    for word in sentence.split():
     if word.lower() in GREET_INPUTS:
      return random.choice(GREET_RESPONSES)
    else:
        return None

In [22]:
def response(user_response): 
  robo1_response=''
  TfidfVec = TfidfVectorizer()
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf) 
  idx = vals.argsort()[0][-2] 
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if (req_tfidf==0): 
    robo1_response = robo1_response+"I am sorry! I don't understand you" 
    return robo1_response
  else: 
    robo1_response = robo1_response+sent_tokens[idx] 
    return robo1_response

In [23]:
set_pro = []
set_cons = []
set_neutral = []
with open('cleaned_data.txt') as file:
    for line in file:
        input_line=line.rstrip()
        texts_p = []
        prediction_input = input_line
        prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
        prediction_input = ''.join(prediction_input)
        texts_p.append(prediction_input)
        prediction_input = tokenizer.texts_to_sequences(texts_p)
        prediction_input = np.array(prediction_input).reshape(-1)
        prediction_input = pad_sequences([prediction_input], input_shape)
        output = model.predict(prediction_input)
        output = output.argmax()
        response_tag = le.inverse_transform([output])[0]
        if response_tag == 'pros':
            set_cons.append(input_line)
        if response_tag == 'cons':
            set_pro.append(input_line)
        if response_tag == 'neutral':
            set_neutral.append(input_line)
        
            
  

2021-12-20 16:01:45 [py.warnings] WARNING: C:\Users\csgoc\AppData\Local\Programs\Python\Python37\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,



In [24]:
p= open("pros.txt", "w")
for element in set_pro:
    p.write(element + "\n")
c= open("cons.txt", "w")
for linr in set_cons:
    c.write(linr + "\n")
n= open("neut.txt", "w")
for pol in set_neutral:
    n.write(pol + "\n")

In [25]:
print(f"Oh hi ! I'm DebaterX ,I Am Programmed To Debate On The Topic {first_line} ,If You Had Enough Of Me , Say Bye")
flag = True
 
import string
while(flag == True):
    texts_p = []
    user_response = input()
    user_response = user_response.lower()
    user_response2 = user_response
    if (user_response != 'bye'):
        if (user_response == 'thanks' or user_response == 'thank you' or user_response == 'thanks for help' or user_response == 'cool'or user_response == 'cya'or user_response == 'good bye'or user_response == 'great'): 
            flag = False
            bye_res1=["Was good talking to you !","Goodbye! Take care","What a great discussion!Let's have another one in a while!","Awww ,why such a early good bye! Lets have another a debate about a differnt topic soon","That was a good fight indeed!I am waiting for another one ,come back as soon as you are ready!"]
            temp1=random.choice(bye_res1)
            print(f"DebaterX: {temp1}")
        else:     
            if (greet(user_response) != None):
                print("DebaterX: "+greet(user_response))
            else: 
                user_response = [letters.lower() for letters in user_response if letters not in string.punctuation]
                user_response = ''.join(user_response)
                texts_p.append(user_response)
                user_response = tokenizer.texts_to_sequences(texts_p)
                user_response = np.array(user_response).reshape(-1)
                user_response = pad_sequences([user_response], input_shape)
                output = model.predict(user_response)
                output = output.argmax()
                response_tag = le.inverse_transform([output])[0]
                if (response_tag == "pros"):
                    f=open('cons.txt','r',errors = 'ignore')
                if (response_tag == "cons"):
                    f=open('pros.txt','r',errors = 'ignore')
                if (response_tag == "neutral"):
                    f=open('neut.txt','r',errors = 'ignore')
                raw_doc = f.read() 
                raw_doc = raw_doc.lower() 
                sent_tokens.append(user_response2)
                word_tokens=word_tokens+nltk.word_tokenize(user_response2)
                final_words=list(set(word_tokens))
                print("DebaterX: ",end="")
                reply = response(user_response2)
                textBlb = TextBlob(reply)
                reply = textBlb.correct()
                print(reply)
                sent_tokens.remove(user_response2)
    else:
        flag=False 
        bye_res=["Was good talking to you !","Goodbye! Take care","What a great discussion!Let's have another one in a while!","Awww ,why such a early good bye! Lets have another a debate about a differnt topic soon","That was a good fight indeed!I am waiting for another one ,come back as soon as you are ready!"]
        temp=random.choice(bye_res)
        print(f"DebaterX: {temp}")

Oh hi ! I'm DebaterX ,I Am Programmed To Debate On The Topic death sentence  ,If You Had Enough Of Me , Say Bye
death sentence is good as it is tit for tat
DebaterX: The death penalty is not a good example of blind justice.
death sentence can teach a man his crime
DebaterX: His lawyers argued that his death sentence violated the U.
 Virginia and that his execution would violate the proscription in the Federal Death Penalty Act that a sentence of death shall not be carried out upon a person who is mentally retarded.
death sentence can be wrong sometim 
DebaterX: When injections go wrong, it can take a long time for a prisoner to die.
killing a man by pen paper is stupidity
DebaterX: Aquinas is saying that certain contents change a bad act killing into a good act killing to repair the violation of justice done by the person killed, and killing a person who has forfeited their natural worthless by killing.
the person who is killing the person is also a killer 
DebaterX: Is a punishment, t

#### 